<a href="https://colab.research.google.com/github/Emivk/RNA-Digits/blob/master/RNAdigit_tf_comet_mejora2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install comet_ml

In [2]:
import comet_ml
comet_ml.init(project_name="rnadigits")

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Input
from tensorflow.keras.optimizers import RMSprop, SGD, Adam
from tensorflow.keras import regularizers
from keras.callbacks import ModelCheckpoint

In [4]:
experiment = comet_ml.Experiment(
    auto_histogram_weight_logging=True,
    auto_histogram_gradient_logging=True,
    auto_histogram_activation_logging=True,
    log_code=True,
)

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.
COMET INFO: Experiment is live on comet.com https://www.comet.com/emivk/rnadigits/62d93f5600e84568a3ddadc93b9cd716



In [5]:
dataset=mnist.load_data()
(x_train, y_train), (x_test, y_test) = dataset

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255  # x_trainv = x_trainv/255
x_test /= 255

num_classes=10
y_trainc = keras.utils.to_categorical(y_train, num_classes)
y_testc = keras.utils.to_categorical(y_test, num_classes)

In [6]:
parameters = {
    "batch_size": 60,
    "epochs": 50,
    "optimizer": "Adam",
    "loss": "categorical_crossentropy",
}

experiment.log_parameters(parameters)

In [7]:
model = Sequential()
model.add(Input(shape=(28,28))) #Flaten no tiene la opcion input_shape por lo tanto se tiene que agregar esta capa
model.add(Flatten()) #Otra forma de aplanar las imagenes
model.add(Dense(200, activation='relu'))
model.add(Dense(100, activation='relu'))
#model.add(Dense(400, activation='relu', kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4))) #Regularizacion L1L2
#model.add(Dropout(0.2)) #Fraccion de enlaces a eliminar
#model.add(Dense(200)) #Capa lineal , transformacion lineal sin funcion de activacion
#model.add(Activation('tanh')) #Se puede agregar despues la funcion de activacion
#model.add(Dense(400, activation='selu', kernel_regularizer=regularizers.L1(0.01) )) #Regularizacion L1
#model.add(Dense(200, activation='elu', kernel_regularizer=regularizers.L2(l2=1e-4)) ) #Regularizacion L2
#model.add(Dense(50,activation='exponential'))
model.add(Dense(10, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 200)               157000    
                                                                 
 dense_1 (Dense)             (None, 100)               20100     
                                                                 
 dense_2 (Dense)             (None, 10)                1010      
                                                                 
Total params: 178110 (695.74 KB)
Trainable params: 178110 (695.74 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [8]:
# specify the path where you want to save the model
filepath = "mejor-modelo1.2.hdf5"

# initialize the ModelCheckpoint callback
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

In [9]:
model.compile(loss=parameters['loss'],optimizer=parameters['optimizer'],metrics=['accuracy'])
model.fit(x_train, y_trainc,
                    batch_size=parameters['batch_size'],
                    epochs=parameters["epochs"],
                    verbose=1,
                    validation_data=(x_test, y_testc),
                    callbacks=[checkpoint])

COMET INFO: Ignoring automatic log_parameter('verbose') because 'keras:verbose' is in COMET_LOGGING_PARAMETERS_IGNORE


Epoch 1/50
 994/1000 [============================>.] - ETA: 0s - loss: 0.2375 - accuracy: 0.9299
Epoch 1: val_loss improved from inf to 0.10788, saving model to mejor-modelo1.2.hdf5
1000/1000 [==============================] - 8s 6ms/step - loss: 0.2365 - accuracy: 0.9302 - val_loss: 0.1079 - val_accuracy: 0.9665
Epoch 2/50
   7/1000 [..............................] - ETA: 8s - loss: 0.0996 - accuracy: 0.9714

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


 998/1000 [============================>.] - ETA: 0s - loss: 0.0947 - accuracy: 0.9706
Epoch 2: val_loss improved from 0.10788 to 0.09386, saving model to mejor-modelo1.2.hdf5
1000/1000 [==============================] - 8s 8ms/step - loss: 0.0947 - accuracy: 0.9706 - val_loss: 0.0939 - val_accuracy: 0.9703
Epoch 3/50
1000/1000 [==============================] - ETA: 0s - loss: 0.0650 - accuracy: 0.9800
Epoch 3: val_loss improved from 0.09386 to 0.07332, saving model to mejor-modelo1.2.hdf5
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0650 - accuracy: 0.9800 - val_loss: 0.0733 - val_accuracy: 0.9761
Epoch 4/50
 993/1000 [============================>.] - ETA: 0s - loss: 0.0487 - accuracy: 0.9839
Epoch 4: val_loss did not improve from 0.07332
1000/1000 [==============================] - 8s 8ms/step - loss: 0.0489 - accuracy: 0.9839 - val_loss: 0.0750 - val_accuracy: 0.9740
Epoch 5/50
 998/1000 [============================>.] - ETA: 0s - loss: 0.0365 - accuracy: 0.

In [11]:
experiment.log_model("MNIST1", "mejor-modelo1.2.hdf5")

{'web': 'https://www.comet.com/api/asset/download?assetId=38adea7faf3340af86955ab2f68b52a2&experimentKey=62d93f5600e84568a3ddadc93b9cd716',
 'api': 'https://www.comet.com/api/rest/v2/experiment/asset/get-asset?assetId=38adea7faf3340af86955ab2f68b52a2&experimentKey=62d93f5600e84568a3ddadc93b9cd716',
 'assetId': '38adea7faf3340af86955ab2f68b52a2'}

In [12]:
experiment.end()

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/emivk/rnadigits/62d93f5600e84568a3ddadc93b9cd716
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [50]                 : (0.9302166700363159, 0.9991666674613953)
COMET INFO:     batch_accuracy [5000]         : (0.11666666716337204, 1.0)
COMET INFO:     batch_loss [5000]             : (3.377595092501906e-08, 2.346312999725342)
COMET INFO:     epoch_duration [50]           : (5.72204496500035, 8.203189859999839)
COMET INFO:     loss [50]                     : (0.003077890258282423, 0.23650723695755005)
COMET INFO:     val_accuracy [50]             : (0.9664999842643738, 0.9832000136375427)
COMET INFO:     val_los